In [1]:
from pathlib import Path
import logging
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.strtree import STRtree
from shapely.geometry import Point

# ───────────────────────────── config ──────────────────────────────
IN_DIR            = Path("data_denoised")     # denoised input
OUT_DIR           = Path("data_binned")       # enriched output
GRID_FILE         = Path("maps/minimalist_coning.geojson")
TIME_BIN_MINUTES  = 60

# ───────────────────────── logging setup ──────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# ───────────────────── other helper functions ─────────────────────
CHUNK = 5_000_000                       # rows per slice

_grid_gdf = gpd.read_file(GRID_FILE)
if "zone_id" not in _grid_gdf.columns:
    _grid_gdf["zone_id"] = np.arange(len(_grid_gdf), dtype="int32")
_grid_gdf = _grid_gdf.to_crs("EPSG:4326")      # WGS-84

def add_zone_id(df: pd.DataFrame, chunk: int = 5_000_000) -> pd.DataFrame:
    n = len(df)
    zone_out = np.full(n, -1, dtype=np.int32)        # -1  → no polygon

    for start in range(0, n, chunk):
        end = min(start + chunk, n)

        # build GeoDataFrame slice
        pts = gpd.GeoDataFrame(
            df.iloc[start:end],
            geometry=gpd.points_from_xy(df.lon.iloc[start:end],
                                        df.lat.iloc[start:end]),
            crs="EPSG:4326"
        )

        joined = gpd.sjoin(
            pts,
            _grid_gdf[["zone_id", "geometry"]],
            how="left",
            predicate="within"
        )

        # first match per original point
        z = (joined
             .groupby(level=0)["zone_id"]
             .first()
             .dropna())                       # drop NaNs (unmatched)

        idx_absolute = z.index.to_numpy(dtype=np.intp)
        zone_out[idx_absolute] = z.to_numpy(dtype=np.int32)

    df["zone_id"] = zone_out
    return df


def add_time_bin(df: pd.DataFrame,
                 minutes: int = TIME_BIN_MINUTES) -> pd.DataFrame:
    mins = df["datetime"].dt.hour * 60 + df["datetime"].dt.minute
    df["time_bin"] = (mins // minutes).astype("int16")
    return df

# ─────────────────────────── main loop ────────────────────────────
def main() -> None:
    grid = gpd.read_file(GRID_FILE)
    if "zone_id" not in grid.columns:
        grid["zone_id"] = np.arange(len(grid), dtype="int32")

    OUT_DIR.mkdir(parents=True, exist_ok=True)

    for f in IN_DIR.glob("*.parquet"):
        logging.info("Processing %s", f)
        df = pd.read_parquet(f)

        logging.info("Adding spatial bin")
        df = add_zone_id(df)         

        # Drop unmatched rows
        before = len(df)
        df = df[df["zone_id"] != -1].reset_index(drop=True)
        dropped = before - len(df)
        logging.info("Dropped %d unmatched rows (zone_id = -1)", dropped)

        logging.info("Adding time bin")
        df = add_time_bin(df)

        print(f"df.colums= {df.columns}")

        out_path = OUT_DIR / f.name
        df.to_parquet(out_path, index=False, compression="snappy")
        logging.info("Wrote %s", out_path)

if __name__ == "__main__":
    main()

/d/hpc/home/jo83525/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
2025-05-27 12:11:12 INFO: Processing data_denoised/20230331.parquet
2025-05-27 12:11:57 INFO: Adding spatial bin
2025-05-27 12:13:31 INFO: Dropped 33 unmatched rows (zone_id = -1)
2025-05-27 12:13:31 INFO: Adding time bin


df.colums= Index(['deviceid', 'date', 'time', 'lon', 'lat', 'datetime', 'device_change',
       'dist_m', 'dt', 'speed_m_s', 'zone_id', 'time_bin'],
      dtype='object')


2025-05-27 12:13:59 INFO: Wrote data_binned/20230331.parquet
2025-05-27 12:13:59 INFO: Processing data_denoised/20230328.parquet
2025-05-27 12:14:54 INFO: Adding spatial bin
2025-05-27 12:16:28 INFO: Dropped 57 unmatched rows (zone_id = -1)
2025-05-27 12:16:28 INFO: Adding time bin


df.colums= Index(['deviceid', 'date', 'time', 'lon', 'lat', 'datetime', 'device_change',
       'dist_m', 'dt', 'speed_m_s', 'zone_id', 'time_bin'],
      dtype='object')


2025-05-27 12:16:55 INFO: Wrote data_binned/20230328.parquet
2025-05-27 12:16:55 INFO: Processing data_denoised/20230327.parquet
2025-05-27 12:17:56 INFO: Adding spatial bin
2025-05-27 12:19:31 INFO: Dropped 24 unmatched rows (zone_id = -1)
2025-05-27 12:19:31 INFO: Adding time bin


...

2025-05-27 12:24:58 INFO: Wrote data_binned/20230329.parquet
2025-05-27 12:24:58 INFO: Processing data_denoised/20230330.parquet
2025-05-27 12:25:25 INFO: Adding spatial bin
2025-05-27 12:27:00 INFO: Dropped 35 unmatched rows (zone_id = -1)
2025-05-27 12:27:00 INFO: Adding time bin


df.colums= Index(['deviceid', 'date', 'time', 'lon', 'lat', 'datetime', 'device_change',
       'dist_m', 'dt', 'speed_m_s', 'zone_id', 'time_bin'],
      dtype='object')


2025-05-27 12:27:28 INFO: Wrote data_binned/20230330.parquet
2025-05-27 12:27:28 INFO: Processing data_denoised/20230402.parquet
2025-05-27 12:27:56 INFO: Adding spatial bin
2025-05-27 12:29:16 INFO: Dropped 27 unmatched rows (zone_id = -1)
2025-05-27 12:29:16 INFO: Adding time bin


df.colums= Index(['deviceid', 'date', 'time', 'lon', 'lat', 'datetime', 'device_change',
       'dist_m', 'dt', 'speed_m_s', 'zone_id', 'time_bin'],
      dtype='object')


2025-05-27 12:29:40 INFO: Wrote data_binned/20230402.parquet


In [2]:
from pathlib import Path
import pandas as pd
import pyarrow.parquet as pq
from pprint import pprint

RAW_DIR = Path("data")
BIN_DIR = Path("data_binned")

def columns_in_file(path):
    return pq.ParquetFile(path).schema_arrow.names

def build_datetime(df):
    if "datetime" in df.columns:
        return df
    if {"date", "time"}.issubset(df.columns):
        ts = pd.to_datetime(df["date"].astype(str) + " " +
                            df["time"].astype(str),
                            errors="coerce")
        df = df.assign(datetime=ts)
    return df.dropna(subset=["datetime"])

def stats(df):
    n_pts = len(df)
    n_dev = df["deviceid"].nunique()
    hrs   = (df["datetime"].max() - df["datetime"].min()
            ).total_seconds()/3600
    return dict(
        n_points=n_pts,
        n_devices=n_dev,
        avg_pts_per_device=round(n_pts/n_dev, 2) if n_dev else 0,
        hours_covered=round(hrs, 2)
    )

def compare(a, b):
    out = {}
    for k in a:
        before, after = a[k], b[k]
        red = round((before - after) / before * 100, 2) if before else None
        out[k] = {"before": before, "after": after, "reduction_%": red}
    return out

def main():
    for raw in sorted(RAW_DIR.glob("*.parquet")):
        binned = BIN_DIR / raw.name
        if not binned.exists():
            print(f"⚠️  missing binned file for {raw.name}")
            continue

        want_cols = {"deviceid", "date", "time", "lon", "lat", "datetime"}
        raw_cols  = want_cols.intersection(columns_in_file(raw))

        df_raw = pd.read_parquet(raw, columns=list(raw_cols))
        df_raw = build_datetime(df_raw)

        df_bin = pd.read_parquet(
            binned,
            columns=["deviceid", "datetime", "lon", "lat"]
        )

        s_raw, s_bin = stats(df_raw), stats(df_bin)
        comp = compare(s_raw, s_bin)

        print(f"\n=== {raw.stem} ===")
        pprint({"raw": s_raw, "binned": s_bin, "comparison": comp})

if __name__ == "__main__":
    main()


/tmp/ipykernel_1722000/710004746.py:17: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ts = pd.to_datetime(df["date"].astype(str) + " " +



=== 20230327 ===
{'binned': {'avg_pts_per_device': 175.36,
            'hours_covered': 24.0,
            'n_devices': 501715,
            'n_points': 87981395},
 'comparison': {'avg_pts_per_device': {'after': 175.36,
                                       'before': 177.66,
                                       'reduction_%': 1.29},
                'hours_covered': {'after': 24.0,
                                  'before': 24.0,
                                  'reduction_%': 0.0},
                'n_devices': {'after': 501715,
                              'before': 751168,
                              'reduction_%': 33.21},
                'n_points': {'after': 87981395,
                             'before': 133453155,
                             'reduction_%': 34.07}},
 'raw': {'avg_pts_per_device': 177.66,
         'hours_covered': 24.0,
         'n_devices': 751168,
         'n_points': 133453155}}


/tmp/ipykernel_1722000/710004746.py:17: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ts = pd.to_datetime(df["date"].astype(str) + " " +



=== 20230328 ===
{'binned': {'avg_pts_per_device': 172.96,
            'hours_covered': 24.0,
            'n_devices': 500313,
            'n_points': 86532152},
 'comparison': {'avg_pts_per_device': {'after': 172.96,
                                       'before': 174.33,
                                       'reduction_%': 0.79},
                'hours_covered': {'after': 24.0,
                                  'before': 24.0,
                                  'reduction_%': 0.0},
                'n_devices': {'after': 500313,
                              'before': 749459,
                              'reduction_%': 33.24},
                'n_points': {'after': 86532152,
                             'before': 130654297,
                             'reduction_%': 33.77}},
 'raw': {'avg_pts_per_device': 174.33,
         'hours_covered': 24.0,
         'n_devices': 749459,
         'n_points': 130654297}}


/tmp/ipykernel_1722000/710004746.py:17: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ts = pd.to_datetime(df["date"].astype(str) + " " +



=== 20230329 ===
{'binned': {'avg_pts_per_device': 174.48,
            'hours_covered': 24.0,
            'n_devices': 507342,
            'n_points': 88523283},
 'comparison': {'avg_pts_per_device': {'after': 174.48,
                                       'before': 178.31,
                                       'reduction_%': 2.15},
                'hours_covered': {'after': 24.0,
                                  'before': 24.0,
                                  'reduction_%': 0.0},
                'n_devices': {'after': 507342,
                              'before': 758892,
                              'reduction_%': 33.15},
                'n_points': {'after': 88523283,
                             'before': 135320914,
                             'reduction_%': 34.58}},
 'raw': {'avg_pts_per_device': 178.31,
         'hours_covered': 24.0,
         'n_devices': 758892,
         'n_points': 135320914}}


/tmp/ipykernel_1722000/710004746.py:17: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ts = pd.to_datetime(df["date"].astype(str) + " " +



=== 20230330 ===
{'binned': {'avg_pts_per_device': 172.11,
            'hours_covered': 24.0,
            'n_devices': 513656,
            'n_points': 88407616},
 'comparison': {'avg_pts_per_device': {'after': 172.11,
                                       'before': 176.74,
                                       'reduction_%': 2.62},
                'hours_covered': {'after': 24.0,
                                  'before': 24.0,
                                  'reduction_%': 0.0},
                'n_devices': {'after': 513656,
                              'before': 768525,
                              'reduction_%': 33.16},
                'n_points': {'after': 88407616,
                             'before': 135827994,
                             'reduction_%': 34.91}},
 'raw': {'avg_pts_per_device': 176.74,
         'hours_covered': 24.0,
         'n_devices': 768525,
         'n_points': 135827994}}


/tmp/ipykernel_1722000/710004746.py:17: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ts = pd.to_datetime(df["date"].astype(str) + " " +



=== 20230331 ===
{'binned': {'avg_pts_per_device': 160.83,
            'hours_covered': 24.0,
            'n_devices': 542576,
            'n_points': 87260420},
 'comparison': {'avg_pts_per_device': {'after': 160.83,
                                       'before': 167.63,
                                       'reduction_%': 4.06},
                'hours_covered': {'after': 24.0,
                                  'before': 24.0,
                                  'reduction_%': 0.0},
                'n_devices': {'after': 542576,
                              'before': 802897,
                              'reduction_%': 32.42},
                'n_points': {'after': 87260420,
                             'before': 134586862,
                             'reduction_%': 35.16}},
 'raw': {'avg_pts_per_device': 167.63,
         'hours_covered': 24.0,
         'n_devices': 802897,
         'n_points': 134586862}}

=== 20230401 ===
{'binned': {'avg_pts_per_device': 157.63,
            'hou